# Top Youtubers Scraper

In [1]:
# Load the libraries
 
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

# Crea listas para almacenar los datos de los streamers, los nombres de los artistas y las categorías
todos_datos_streamers = []
todos_nombres_artistas = []
todas_categorias = []
todas_suscriptores = []
todos_paises = []
todos_visitas = []
todos_likes = []
todos_comentarios = []
todos_links = []

# Función para convertir las cifras con 'M' en millones y 'K' en miles
def convert_number(number_str):
    if 'M' in number_str:
        return float(number_str.replace('M', '')) * 1e6
    elif 'K' in number_str:
        return float(number_str.replace('K', '')) * 1e3
    else:
        return float(number_str)
    
def normalize_categories(categoria_text):

    # Diccionario para mapear categorías individuales a sus equivalentes normalizados
    category_mapping = {
        'Diseño/arteDIY': ['Diseño/arte', 'DIY'],
        'BellezaModa': ['Belleza', 'Moda'],
        'FitnessSalud y autoayuda': ['Fitness', 'Salud y autoayuda'],
        'Comida y bebidaSalud': ['Comida y bebida', 'Salud'],
        'ViajesEspectáculos': ['Viajes', 'Espectáculos'],
        'Diseño/arteBelleza': ['Diseño/arte', 'Belleza'],
        'ASMRComida': ['ASMR', 'Comida'],
        'PelículasVideojuegos': ['Películas', 'Videojuegos'],
        'PelículasAnimación': ['Películas', 'Animación'],
        'AnimaciónHumor': ['Animación', 'Humor'],
        'PelículasJuguetes': ['Películas', 'Juguetes'],
        'PelículasHumor': ['Películas', 'Humor'],
        'VideojuegosJuguetes': ['Videojuegos', 'Juguetes'],
        'AnimaciónVideojuegos': ['Animación', 'Videojuegos'],
        'Música y baileHumor': ['Música y baile', 'Humor'],
        'DIY y Life HacksJuguetes': ['DIY y Life Hacks', 'Juguetes'],
        'EducaciónJuguetes': ['Educación', 'Juguetes'],
        'JuguetesCoches y vehículos': ['Juguetes', 'Coches y vehículos'],
        'Música y baileJuguetes': ['Música y baile', 'Juguetes'],
        'AnimaciónHumorJuguetes': ['Animación', 'Humor', 'Juguetes'],
        'Comida y bebidaJuguetes': ['Comida y bebida', 'Juguetes'],
        'JuguetesDIY y Life Hacks': ['Juguetes', 'DIY y Life Hacks'],
        'AnimaciónJuguetes': ['Animación', 'Juguetes'],
        'VideojuegosHumor': ['Videojuegos', 'Humor'],
        'ASMRComida y bebida': ['ASMR', 'Comida y bebida'],
        'Comida y bebidaSalud y autoayuda': ['Comida y bebida', 'Salud y autoayuda'],
        'Diseño/arteDIY y Life Hacks': ['Diseño/arte', 'DIY y Life Hacks'],
        'Música y bailePelículas': ['Música y baile', 'Películas'],
        'Música y baileAnimación': ['Música y baile', 'Animación']
    }


    # Dividir el texto en categorías individuales
    categorias = [c.strip() for c in categoria_text.split('•')]

    # Inicializar una lista para almacenar categorías normalizadas
    normalized_categorias = []

    for categoria in categorias:
        if categoria in category_mapping:
            # Si la categoría está en el mapeo, agregar las categorías normalizadas
            normalized_categorias.extend(category_mapping[categoria])
        else:
            # Si la categoría no está en el mapeo, agrégala como está
            normalized_categorias.append(categoria)

    # Unir las categorías normalizadas con comas
    return ', '.join(normalized_categorias)

# Itera a través de las páginas de 1 a 2 (para pruebas, puedes aumentar a 10)
for pagina in range(1, 21):
    # Construye la URL con el valor de la página actual
    url = f'https://hypeauditor.com/es/top-youtube/?p={pagina}'
    
    # Realiza una solicitud HTTP para obtener el contenido de la página
    response = requests.get(url)
    
    # Comprueba si la solicitud fue exitosa
    if response.status_code == 200:
        # Parsea el contenido HTML de la página con BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Encuentra la tabla principal para el ranking
        tabla = soup.find('div', class_='table')

        # Encuentra todas las filas en la tabla del ranking
        filas_ranking = tabla.find_all('div', class_='row-cell rank')

        # Encuentra todas las filas en la tabla de los nombres de los artistas
        filas_artistas = tabla.find_all('div', class_='contributor__name-content')

        # Encuentra todas las filas en la tabla de las categorías
        filas_categorias = tabla.find_all('div', class_='row-cell category')

        # Load followers variable
        filas_suscriptores = tabla.find_all('div', class_='row-cell subscribers')

        # Load the country variable
        filas_paises = tabla.find_all('div', class_='row-cell audience')

        # Load average views
        filas_visitas = tabla.find_all('div', class_='row-cell avg-views')

        # Load average likes
        filas_likes = tabla.find_all('div', class_='row-cell avg-likes')

        # Load average comments
        filas_comentarios = tabla.find_all('div', class_='row-cell avg-comments')

        # Load the channel link

        filas_link = tabla.find_all('a', class_='contributor-link')

        # Procesa las categorías y separa en función de las categorías específicas
        for i, categoria in enumerate(filas_categorias):
            categoria_text = categoria.text.strip()
            categorias_normalizadas = normalize_categories(categoria_text)
            todas_categorias.append({'categorias': categorias_normalizadas})

        # Crea listas para almacenar los datos de los streamers, los nombres de los artistas y las categorías de esta página
        datos_streamers = [{'streamer': fila_ranking.text} for fila_ranking in filas_ranking]
        nombres_artistas = [{'artista': fila_artista.text} for fila_artista in filas_artistas]
        suscriptores = [{'suscriptores': fila_suscriptores.text} for fila_suscriptores in filas_suscriptores]
        paises = [{'country': fila_pais.text if fila_pais.text else 'Unknown'} for fila_pais in filas_paises]
        visitas = [{'visitas': fila_visitas.text} for fila_visitas in filas_visitas]
        likes = [{'likes': fila_likes.text} for fila_likes in filas_likes]
        comentarios = [{'comentarios': fila_comentarios.text} for fila_comentarios in filas_comentarios]
        datos_link = [{'link': fila_link['href']} for fila_link in filas_link]

        # Agrega los datos de esta página a las listas globales
        todos_datos_streamers.extend(datos_streamers)
        todos_nombres_artistas.extend(nombres_artistas)
        todas_suscriptores.extend(suscriptores)
        todos_paises.extend(paises)
        todos_visitas.extend(visitas)
        todos_likes.extend(likes)
        todos_comentarios.extend(comentarios)
        todos_links.extend(datos_link)


# Crea un DataFrame de pandas con los datos recopilados de todas las páginas
df = pd.DataFrame({'Rank': [item['streamer'] for item in todos_datos_streamers],
                   'Username': [item['artista'] for item in todos_nombres_artistas],
                   'Categories': [item['categorias'] for item in todas_categorias],
                   'Suscribers': [convert_number(item['suscriptores']) for item in todas_suscriptores],
                   'Country': [item['country'] for item in todos_paises],
                   'Visits': [convert_number(item['visitas']) for item in todos_visitas],
                   'Likes': [convert_number(item['likes']) for item in todos_likes],
                   'Comments': [convert_number(item['comentarios']) for item in todos_comentarios],
                   'Links': [item['link'] for item in todos_links]})

# Reemplaza 'False' por 'Unknown' en la columna 'Country'
df['Country'] = df['Country'].replace('false', 'Unknown')

In [4]:
# Export the df into csv. Encode to utf-8 to avoid problems with accents

df.to_csv('../docs/youtubers_df.csv', index=False, encoding='utf-8')